# Implementation of some features based on POS patterns and sentiment

In [56]:
import nltk as nl
import pandas as pd
import numpy as np
import twython
import re
from nltk.sentiment import SentimentAnalyzer # sentiment analysis
from nltk.tokenize import TweetTokenizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer # sentiment analysis VADER
from nltk.tokenize import sent_tokenize #sentence tokenizer
import itertools
#nl.download('vader_lexicon')
#nl.download('averaged_perceptron_tagger') for POS
#nl.download('punkt')

In [68]:
# TrainingSet = pd.read_json("train1703.jsonl", lines = True)
TrainingSet = pd.read_json("stopclickbait.json", lines = True)

In [69]:
#only works for postTitle
def removeRT (content):
    content=content[0]
    if bool(re.search(r'\bRT ', content)):
        content = re.sub(r"\bRT ","",content)
    return([content])

TrainingSet["postText"] = TrainingSet["postText"].apply(removeRT)

AttributeError: 'DataFrame' object has no attribute 'eys'

In [114]:
features = TrainingSet[['id']].copy() # implemented features will be added to this matrix.

In [115]:
len(features.keys())

1

# Sentiment scores

In [105]:
#Sentiment Score VADER : of post title, target title and their differences (more info on https://github.com/cjhutto/vaderSentiment)
sid = SentimentIntensityAnalyzer()
postText = TrainingSet["postText"].tolist()
targetText = TrainingSet["targetTitle"].tolist()

#Add sentiment features to feature table
countsDF = pd.DataFrame(columns = ["sentimentPostText","sentimentTargetText","difsentimentPostTarget"])

countsDF["sentimentPostText"] = [sid.polarity_scores(sentence[0])["compound"] for sentence in postText] # vector with the sentiment score of each post text
countsDF["sentimentTargetText"] = [sid.polarity_scores(sentence)["compound"] for sentence in targetText] #vector with sentiment score of target Text
countsDF["difsentimentPostTarget"]= countsDF["sentimentPostText"] - countsDF["sentimentTargetText"]
sentiment=pd.concat([features,countsDF],axis=1)
features = pd.concat([features,countsDF],axis=1)

In [40]:
len(features.keys())

34

# POS patterns

In [72]:
#Getting POS tags
#join everything
total=TrainingSet['postText'].apply(", ".join)
total = ' '.join(total.tolist())
tokenizer =  TweetTokenizer()
posTags = pd.DataFrame(nl.pos_tag(tokenizer.tokenize(total)))
posCategories = posTags[1].value_counts().index
posCategories=posCategories.tolist() # contains POS tags found in our dataset
#bigrams=list(itertools.permutations(posCategories, 2))
#trigrams=list(itertools.permutations(posCategories, 3))

In [73]:
def POSFrequency (sentence,first):
    count=0
    if type(sentence) is not str:
        sentence=sentence[0]
    tokenizer =  TweetTokenizer()
    posTags = nl.pos_tag(tokenizer.tokenize(sentence))
    for (w1,t1) in posTags:
        if (t1==first):
            count=count+1
    return(count)

In [74]:
posPatternsDF=pd.DataFrame()
selectedCAtegories = ["NNP","NN","IN","NNS","DT",":","VBZ","VB",",","RB","VBP","PRP","VBN","PRP$","MD","POS","WRB","WP","''","JJS","JJR","$","WDT","(",")","RBS","FW","EX","UH"]
#for tag in posCategories:
for tag in selectedCAtegories:
    counts = TrainingSet['postText'].apply(POSFrequency,args=(tag,))
    colname="POS_"+tag
    countsDF = pd.DataFrame(columns = [colname])
    countsDF[colname]=counts
    posPatternsDF=pd.concat([posPatternsDF,countsDF],axis=1)

In [117]:
len(countsDF.keys())

3

In [75]:
#POS Bigram
#best2grams = [('NNP','NNP'),('NNP','VBZ'),('IN','NNP'),('NNP',"."),('PRP','VBP'),('NN','IN'),('IN','NN'),('NNP','NNS'),('IN','JJ'),('NNP','POS'),('NN','NN'),('NN','NNP'),('NNP','VBD'),('VBN','IN'),('JJ','NNP'),('DT','NN'),('CD','NN'),('CD','NNP'),('CD','NNS'),('CD','JJ'),('CD','JJR'),('CD','JJS')]
best2grams=[("NNP","NN"),("NNP","IN"),("NNP","NNS"),("NNP",":"),("NNP","VBZ"),("NNP","VB"),("NNP","VBD"),("NNP","."),("NNP","CD"),("NNP",","),
("NNP","TO"),("NNP","VBG"),("NNP","VBP"),("NNP","PRP"),("NNP","CC"),("NNP","PRP$"),("NNP","WRB"),("NNP","WP"),("NNP","$"),("NNP","WDT"),("NNP",")"),("NNP","FW"),("NN","NNP"),
("NN","IN"),("NN","NNS"),("NN","DT"),("NN",":"),("NN","VBZ"),("NN","VBD"),("NN","."),("NN","CD"),("NN",","),("NN","TO"),("NN","VBG"),("NN","VBP"),
("NN","MD"),("NN","WP"),("NN","''"),("NN","NNPS"),("NN","WDT"),("NN","("),("NN","RBR"),("NN","EX"),("IN","NNP"),("IN","NN"),("IN","JJ"),
("IN","NNS"),("IN","DT"),("IN","CD"),("IN",","),("IN","RB"),("IN","CC"),("IN","POS"),("IN","WRB"),("IN","JJS"),("IN","$"),("JJ","DT"),
("JJ","VBZ"),("JJ","VB"),("JJ","VBD"),("JJ",","),("JJ","RB"),("JJ","CC"),("JJ","PRP$"),("JJ","''"),("JJ","JJR"),("JJ","$"),
("JJ","("),("JJ",")"),("NNS","IN"),("NNS","DT"),("NNS",":"),("NNS","."),("NNS",","),("NNS","TO"),("NNS","PRP"),("NNS","VBN"),
("NNS","CC"),("NNS","POS"),("NNS","WP"),("NNS","''"),("NNS","NNPS"),("NNS","$"),("NNS","WDT"),("NNS","("),("NNS",")"),
("DT","NNP"),("DT","NN"),("DT","JJ"),("DT","VBZ"),("DT","VB"),("DT","CD"),("DT","RB"),("DT","VBP"),("DT","PRP"),("DT","VBN"),("DT","PRP$"),("DT","MD"),("DT","JJS"),
("DT","$"),("DT","("),("DT","RBS"),(":","NNP"),(":","NN"),(":","IN"),(":","JJ"),(":","NNS"),(":","DT"),(":","VBZ"),(":","VB"),
(":","VBP"),(":","PRP"),(":","VBN"),(":","WRB"),(":","WP"),(":","("),("VBZ","NNP"),("VBZ","NN"),
("VBZ","IN"),("VBZ",":"),("VBZ","VBD"),("VBZ","CD"),("VBZ","PRP"),("VBZ","VBN"),("VBZ","WRB"),("VBZ","JJS"),("VBZ","JJR"),("VBZ","$"),("VBZ","("),
("VBZ","RBR"),("VB","JJ"),("VB","DT"),("VB",":"),("VB","VBZ"),("VB","VBD"),("VB","."),("VB","TO"),("VB","VBG"),("VB","PRP"),("VB","PRP$"),("VB","MD"),("VB","RP"),("VB","WP"),("VB","JJR"),
("VB","$"),("VB","PDT"),("VB","FW"),("VBD","NNP"),("VBD","NN"),("VBD","IN"),("VBD","DT"),("VBD","VBN"),("VBD","PRP$"),("VBD","WRB"),("VBD","RP"),("VBD","WP"),("VBD","JJS"),("VBD","RBR"),
(".","NNP"),(".","NN"),(".","NNS"),(".",":"),(".","VBZ"),(".","VB"),(".","CD"),(".",","),(".","RB"),(".","CC"),(".","WP"),(".","''"),(".","NNPS"),(".",")"),
(".","EX"),("CD","NNP"),("CD","IN"),("CD","JJ"),("CD","NNS"),("CD",":"),("CD","."),("CD",","),
("CD","RB"),("CD","TO"),("CD","JJS"),("CD","RBS"),(",","NNP"),(",","NN"),(",","NNS"),(",","DT"),(",","VBZ"),
(",","TO"),(",","VBG"),(",","PRP"),(",","VBN"),(",","WRB"),(",","WP"),(",","JJS"),("RB","IN"),
("RB","DT"),("RB","VBZ"),("RB","VB"),("RB","VBD"),("RB","."),("RB","CD"),("RB",","),("RB","TO"),
("RB","VBG"),("RB","PRP"),("RB","WRB"),("RB","WP"),("RB","("),("RB",")"),("TO","NNP"),("TO","NN"),("TO","IN"),("TO","JJ"),("TO","VBZ"),("TO","POS"),
("TO","JJR"),("TO","$"),("VBG","NN"),("VBG","IN"),("VBG","DT"),("VBG",":"),("VBG","VBD"),("VBG",","),("VBG","RB"),("VBG","PRP"),("VBG","CC"),("VBG","MD"),("VBG","WRB"),("VBG","JJR"),("VBG","$"),("VBP","NNP"),("VBP","NN"),
("VBP","NNS"),("VBP","DT"),("VBP","VB"),("VBP","RB"),("VBP","TO"),("VBP","$"),("VBP","RBS"),("PRP","DT"),
("PRP","VB"),("PRP","VBD"),("PRP","RB"),("PRP","VBP"),("PRP","CC"),("PRP","MD"),("PRP","RP"),("PRP","EX"),("VBN","NN"),("VBN","IN"),("VBN","JJ"),("VBN","DT"),("VBN",":"),("VBN","VBD"),("VBN","CD"),("VBN","TO"),
("VBN","VBP"),("VBN","PRP"),("VBN","CC"),("VBN","RP"),("VBN","''"),("VBN","JJR"),("CC","NNP"),("CC","NNS"),("CC","VBZ"),("CC","VB"),("CC","VBP"),("CC","PRP"),("CC","POS"),("CC","WRB"),("PRP$","NN"),("MD","JJ"),("MD","DT"),("MD","VB"),("MD","RB"),("MD","PRP"),("MD","PRP$"),("MD","("),("MD","RBR"),("POS","NN"),("POS","NNS"),("POS","DT"),("POS",":"),("POS","MD"),("POS","WDT"),("WRB","NNP"),("WRB","NN"),("WRB","IN"),("WRB","JJ"),("WRB","NNS"),("WRB","DT"),("WRB","TO"),("WRB","VBG"),("WRB","VBP"),("WRB","PRP"),
("WRB","MD"),("WRB","''"),("WRB","JJR"),("WRB","NNPS"),("WRB","EX"),("RP","IN"),("RP","DT"),("RP",":"),("RP","."),("RP","CD"),("RP",","),("RP","WRB"),("RP","WP"),("RP","''"),("RP","JJS"),("RP","JJR"),("WP","NNP"),("WP","NN"),("WP","VBZ"),("WP","CD"),("WP","TO"),("WP","VBG"),("WP","VBP"),("WP","PRP"),("WP","PRP$"),("WP","MD"),("''","NN"),("''","JJ"),("''","NNS"),("''","DT"),("''","MD"),("JJS","NNS"),("JJS","CD"),("JJS","RB"),
("JJS","CC"),("JJR","IN"),("JJR","JJ"),("JJR","VBZ"),("JJR","RB"),("NNPS","IN"),("NNPS","VB"),("NNPS","TO"),("$","CD"),("WDT","VBP"),("WDT","MD"),("(","NN"),
("(","JJ"),("(","NNS"),("(","DT"),("(","RB"),("(","VBG"),("(","CC"),(")","NNP"),(")","IN"),(")","JJ"),(")","VBP"),("RBS","JJ"),("RBR","NNS"),("RBR",":"),("PDT","IN"),("FW","NNP"),("EX",":"),("EX","VBP"),("EX","MD"),("WP$","NN")]
def POSBigram(sentence,first,second):
    count=0
    if type(sentence) is not str:
        sentence=sentence[0]
    tokenizer =  TweetTokenizer()
    posTags = nl.pos_tag(tokenizer.tokenize(sentence))
    for (w1,t1), (w2,t2) in nl.bigrams(posTags):
        if (t1==first and t2 == second):
            count=count+1
    return(count)

In [76]:
i=1
for pairs in best2grams:
    print(i)
    counts = TrainingSet['postText'].apply(POSBigram,args=(pairs[0],pairs[1]))
    colname="POS2_"+pairs[0]+"_"+pairs[1]
    countsDF = pd.DataFrame(columns = [colname])
    countsDF[colname]=counts
    posPatternsDF=pd.concat([posPatternsDF,countsDF],axis=1)
    i=i+1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [41]:
len(features.keys())

34

In [77]:
#POS Trigram
#best3grams = [('NNP','NNP','VBZ'),('NN','IN','NNP'),('IN','NNP','NNP'),('NNP','NNP','NNP'),('NNP','NNP','NN'),('NNP','NN','NN')]
best3grams = [("NNP","NNP","VBZ"),("NN","IN","NNP"),("IN","NNP","NNP"),("NNP","NNP","NNP"),("NNP","NNP","NN"),("NNP","NN","NN")]
def POSTrigram(sentence,first,second,third):
    count=0
    if type(sentence) is not str:
        sentence=sentence[0]
    tokenizer =  TweetTokenizer()
    posTags = nl.pos_tag(tokenizer.tokenize(sentence))
    for (w1,t1), (w2,t2),(w3,t3) in nl.trigrams(posTags):
        if (t1==first and t2 == second and t3 == third):
            count=count+1
    return(count)

In [78]:
for tris in best3grams:
    counts = TrainingSet['postText'].apply(POSTrigram,args=(tris[0],tris[1],tris[2]))
    colname="POS3_"+tris[0]+"_"+tris[1]+"_"+tris[2]
    countsDF = pd.DataFrame(columns = [colname])
    countsDF[colname]=counts
    posPatternsDF=pd.concat([posPatternsDF,countsDF],axis=1)

In [82]:
def StartsWithNumber(sentence):
    if type(sentence) is not str:
        sentence=sentence[0]
    tokenizer =  TweetTokenizer()
    posTags = nl.pos_tag(tokenizer.tokenize(sentence))
    if posTags[0][1]=="CD":
        start=1
    else:
        start=0

    return(start)

In [83]:
def StartsWith5WIH(sentence):
    if type(sentence) is not str:
        sentence=sentence[0]
    tokenizer =  TweetTokenizer()
    token_sentence=tokenizer.tokenize(sentence)
    WIH5 = ["where","what","when","which","how","why","can","could","will"]
    firstWord=token_sentence[0].lower()
    if firstWord in WIH5:
        start=1
    else:
        start=0
    return(start)

In [84]:
def containsThisNN(sentence):
    count=0
    if type(sentence) is not str:
        sentence=sentence[0]
    tokenizer =  TweetTokenizer()
    posTags = nl.pos_tag(tokenizer.tokenize(sentence))
    for (w1,t1), (w2,t2) in nl.bigrams(posTags):
        if (w1.lower()=="this" and t2 in ["NN","NNP","NNS"]):
            count=count+ 1
        elif (w1.lower()=="these" and t2 in ["NN","NNP","NNS"]):
            count=count +1 
    return(count)

In [120]:
countsDF = pd.DataFrame(columns = ["StartsWithNumber","StartsWith5WIH","containsThisNN"])
countsDF["StartsWithNumber"]=TrainingSet['postText'].apply(StartsWithNumber)
countsDF["StartsWith5WIH"]=TrainingSet['postText'].apply(StartsWith5WIH)
countsDF["containsThisNN"]=TrainingSet['postText'].apply(containsThisNN)
posPatternsDF=pd.concat([posPatternsDF,countsDF],axis=1)
features = pd.concat([features,posPatternsDF],axis=1)

In [122]:
len(features.keys())

401

# Number of Tokens

In [86]:
#retrieves number of tokens in a content, using a nltk tokenizer especially suited for twitter data. punctuation is NOT removed
def numTwitterTokens(content):
    tokenizer =  TweetTokenizer()
    if content is None:
        return -1
    elif not content:
        return -1
    elif type(content) is str:
        content_no_punc = tokenizer.tokenize(content)
        if(len(content_no_punc)==0):
            return -1
        else:
            return len(content_no_punc)
    else:
        content_no_punc = tokenizer.tokenize(content[0])
        if(len(content_no_punc)==0):
            return -1
        else:
            return len(content_no_punc)

In [87]:
# Other Features
def useOfSuperlative(content):
    count=0
    if type(content) is not str:
        content=content[0]
    content=content.lower()
    tokenizer =  TweetTokenizer()
    posTags = nl.pos_tag(tokenizer.tokenize(content))
    for (w1,t1) in posTags:
        if t1 == "JJS":
            count=count+1
    return(count)

In [88]:
#Presence of Question Mark
def presenceQM(content):
    count=0
    if type(content) is not str:
        content=content[0]
    if "?" in content:
        count=1
    return(count)

In [89]:
#Presence of Exclamation
def presenceExc(content):
    count=0
    if type(content) is not str:
        content=content[0]
    if "!" in content:
        count=1
    return(count)

In [90]:
def numberUpperCaseWords(content):
    if type(content) is not str:
        content=content[0]
    count = len(re.findall(r'\b([A-Z]{2,})\b', content))
    return(count)

In [125]:
contentPostDF = pd.DataFrame(columns = ["nrTokens","useOfSuperlative","presenceQM","presenceExc","numberUpperCaseWords"])
contentPostDF["nrTokens"]=TrainingSet['postText'].apply(numTwitterTokens)
contentPostDF["useOfSuperlative"]=TrainingSet['postText'].apply(useOfSuperlative)
contentPostDF["presenceQM"]=TrainingSet['postText'].apply(presenceQM)
contentPostDF["presenceExc"]=TrainingSet['postText'].apply(presenceExc)
contentPostDF["numberUpperCaseWords"]=TrainingSet['postText'].apply(numberUpperCaseWords)
features = pd.concat([features,contentPostDF],axis=1)

In [126]:
len(features.keys())

406

## Informality Measures (from Biyani et al.)

In [24]:
def processBodyArticle(content):
    
    content= [sentence.replace("Â\xa0", "") for sentence in content]
    content=[i for i in content if i != ""]
    content= " ".join(content)
    return(content)

In [25]:
#CL Score :0.0588 L - 0.296S -15.8 where L = avg nr of letters per 100 words, S=avg number of sentences per 100 words
#content is body of the articles
def computeInformalityScores(content):
    content = processBodyArticle(content)
    if content!="":
        sent_tokenize_list = sent_tokenize(content)
        nrSentences=len(sent_tokenize_list)
        words = re.findall(r'[a-zA-Z]+', content)
        longWords = [w for w in words if len(w)>6]
        nrLongWords = len(longWords)
        nrWords=len(words)
        nrLetters = len("".join(words))
        L = nrLetters*100/nrWords
        S = nrSentences*100/nrWords
        CLScore = 0.0588*L - 0.296*S -15.8
        RIX = nrLongWords/nrSentences
        LIX = nrWords/nrSentences + 100*nrLongWords/nrWords
        res=[CLScore,RIX,LIX]
    else:
        res = [-1,-1,-1]
    
    return res
    

In [26]:
def POSExistence(posTags,tags):
    count=0
    for (w1,t1) in posTags:
        if (t1 in tags):
            count=count+1
    return(count)

def computeFormMeasure(article):
    article = processBodyArticle(article)
    if article !="":
        #tokenization
        tokenizer =  TweetTokenizer()
        posTags = nl.pos_tag(tokenizer.tokenize(article))
        
        
        nrNouns = POSExistence(posTags,["NN","NNP","NNS"])
        nrAdj = POSExistence(posTags,["JJ","JJR","JJS"])
        nrPrep = POSExistence(posTags,["IN","TO"])
        nrArtic=POSExistence(posTags,["DT"])
        nrPron = POSExistence(posTags,["PRP","PRP$"])
        nrVerb = POSExistence(posTags,["VB","VBD","VBG","VBN","VBP","VBZ"])
        nrAdver = POSExistence(posTags,["RB","RBR","RBS","WRB"])
        nrInter = POSExistence(posTags,["UH"])

        measure = (nrNouns+nrAdj+nrPrep+nrArtic-nrPron-nrVerb-nrAdver-nrInter+100)/2
    else:
        measure=-10000000
    return measure
    

In [99]:
features.keys()

Index(['id', 'POS_NNP', 'POS_NN', 'POS_IN', 'POS_NNS', 'POS_DT', 'POS_:',
       'POS_VBZ', 'POS_VB', 'POS_,',
       ...
       'porter_zimbabwean', 'porter_zodiac', 'porter_zone', 'porter_zoo',
       'porter_~', 'porter_£', 'porter_–', 'porter_—', 'porter_’', 'porter_€'],
      dtype='object', length=3728)

In [108]:
#join informality measures to features data frame
informDF = pd.DataFrame(columns = ["inform_CLScore","inform_RIX","inform_LIX","inform_fmeas"])
informalityMeasures=TrainingSet['targetParagraphs'].apply(computeInformalityScores)
informDF["inform_CLScore"]=[metric[0] for metric in informalityMeasures]
informDF["inform_RIX"]=[metric[1] for metric in informalityMeasures]
informDF["inform_LIX"]=[metric[2] for metric in informalityMeasures]
informDF["inform_fmeas"]=TrainingSet['targetParagraphs'].apply(computeFormMeasure)
features=pd.concat([features,informDF],axis=1)

NameError: name 'informDF' is not defined

## Unigrams and Bigrams from PostTitles

In [92]:
from collections import Counter
from nltk.util import ngrams
from nltk.stem import *
from sklearn.feature_extraction.text import CountVectorizer # count unigrams

In [128]:
#Retrieve words from postTitles
postTitles = TrainingSet["postText"].tolist()

def stemmingPostTitles(postTitles,stem):
    tokenizer =  TweetTokenizer()
    if (stem=="porter"):
        stemmer = nl.PorterStemmer() # Porter Stemming
    elif (stem == "snowball"):
        stemmer = SnowballStemmer("english")
    elif (stem == "lancaster"):
        stemmer = LancasterStemmer()
    
    number_pattern = re.compile(r'\d+')
    corpus=[]
    for title in postTitles:
        title = title[0]
        
        title=title.lower()
        tokens = tokenizer.tokenize(title)
        stem_tokens = [stemmer.stem(t) for t in tokens ]
        stem_title = " ".join(stem_tokens)
        stem_title = number_pattern.sub('[N]', stem_title)
        corpus.append(stem_title)
    return(corpus)

def getUniAndBigramsFeatures(postTitles,stem,pres,nr):
    corpusPostTitles=stemmingPostTitles(postTitles,stem)
    cv_kw = CountVectorizer(token_pattern='\\S+', ngram_range=(nr, nr), min_df=10,  # at least 10 occurences
                     binary=pres, dtype=np.uint8)
    X_kw = cv_kw.fit_transform(corpusPostTitles)
    df=pd.DataFrame(X_kw.toarray())
    featurenames = [re.sub(" ","_",name) for name in cv_kw.get_feature_names()]
    df.columns = [stem+"_"+feature for feature in featurenames]
    return(df)
    

In [129]:
uniFeat=getUniAndBigramsFeatures(postTitles,"porter",True,1) # atm we are only using porter stemmer, we can see if changing the stemmer has any impact on the result.

In [130]:
# bigFeat= getUniAndBigramsFeatures(postTitles,"porter",False,2)
len(uniFeat.keys())

3322

In [131]:
features = pd.concat([features,uniFeat],axis=1)
#features = pd.concat([features,bigFeat],axis=1)
features.keys()

Index(['id', 'POS_NNP', 'POS_NN', 'POS_IN', 'POS_NNS', 'POS_DT', 'POS_:',
       'POS_VBZ', 'POS_VB', 'POS_,',
       ...
       'porter_zimbabwean', 'porter_zodiac', 'porter_zone', 'porter_zoo',
       'porter_~', 'porter_£', 'porter_–', 'porter_—', 'porter_’', 'porter_€'],
      dtype='object', length=3728)

In [186]:
# Fran - Testing 
# col_names = pd.DataFrame(features.columns.values)
# dup_idx = pd.DataFrame.duplicated(col_names)
# dup_idx[dup_idx == True]
len(features.keys())

3728

## Classification - some experiments

In [188]:
# features.to_csv("features_barbara_all.csv")
features.to_csv("features_barbara_all_stopclickbait.csv", index=False, encoding='utf-8')

In [ ]:
# Classification: finding classifiers 
from sklearn.linear_model import LogisticRegressionCV

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif
from sklearn.naive_bayes import GaussianNB

In [ ]:
##Getting the Labels
testLabelsFile = 'traintruth1703.jsonl'
Labels = pd.read_json(testLabelsFile, lines=True)
Labels.loc[Labels.iloc[:,1] == "no-clickbait", "truthClass"] = -1
Labels.loc[Labels.iloc[:,1] == "clickbait", "truthClass"] = 1

#Add labels to each of the tables corresponding to a feature group
sentimentWithLabel= pd.merge(sentiment, Labels[["id","truthClass"]], on = "id")
posPatternsDF.insert(0, "id", TrainingSet["id"].tolist())
posPatternsDFWithLabel = pd.merge(posPatternsDF, Labels[["id","truthClass"]], on = "id")
posPatternsDFWithLabel.to_csv("posPatterns_ALL_SVM.csv")
contentPostDF.insert(0, "id", TrainingSet["id"].tolist())
contentPostDFWithLabel = pd.merge(contentPostDF, Labels[["id","truthClass"]], on = "id")#
informDF.insert(0, "id", TrainingSet["id"].tolist())
informDFWithLabel = pd.merge(informDF, Labels[["id","truthClass"]], on = "id")
uniFeat.insert(0, "id", TrainingSet["id"].tolist())
uniDFWithLabel = pd.merge(uniFeat,Labels[["id","truthClass"]], on = "id")
#bigFeat.insert(0, "id", TrainingSet["id"].tolist())
#biDFWithLabel = pd.merge(bigFeat,Labels[["id","truthClass"]], on = "id")
uniDFWithLabel.to_csv("unigrams_select250_Log.csv")
#features = pd.merge(features,Labels[["id","truthClass"]],on="id")

In [ ]:
#readaptation of Nirmal's function
def getAccuracy(classifier, featuresel, numFeat, allfeatures, featurelist):
    #Choose Classifier
    if(classifier == "SVM"):
        clf = SVC()
    elif(classifier == "Log"):
        clf = LogisticRegressionCV()
    elif(classifier == "RandomForest"):
        clf = RandomForestClassifier(criterion = 'entropy')
    elif(classifier == "XgBoost"):
        clf = GradientBoostingClassifier()
    elif(classifier == "NaiveBayes"):
        clf = GaussianNB()
    
    

        
    # select features we want
    x = [col for col in allfeatures if col in featurelist]
    x = allfeatures[x]
    #print(x)
    # normalise features
    featNorm = x.copy()
    for k in range(len(featNorm.columns)):
        m = np.mean(x.iloc[:,k])
        s = np.std(x.iloc[:,k])
        featNorm.iloc[:,k] = (x.iloc[:,k]-m)/s
    
    featNorm = featNorm.iloc[:,1:k]
    #print(featNorm)
    #print(featNorm)
    # Labels
    y = allfeatures.iloc[:,-1]
    
    if(featuresel== "chi"):
        X_new = SelectKBest(chi2, k=numFeat).fit_transform(featNorm, y)
    elif(featuresel == "f_classif"):
        X_new = SelectKBest(f_classif, k=numFeat).fit_transform(featNorm, y)
    elif(featuresel == "mutual_info"):
        X_new = SelectKBest(mutual_info_classif, k=numFeat).fit_transform(featNorm, y)
    else:
        X_new = featNorm
    #Classifier resuts
    scores = cross_val_score(clf, X_new, y, cv=10)
    accuracy = [scores.mean(), scores.std()]
    return(accuracy)

In [ ]:
def useNirmalFunctionEfficiently(classifier, featuresel, numFeat,df):
    accuracy = getAccuracy(classifier, featuresel, numFeat, df, df.iloc[:,range(len(df.columns)-1)])
    return(accuracy)

# Initial analysis - using Log with just one group of features

In [ ]:
#sentiment
useNirmalFunctionEfficiently("Log","none",0,sentimentWithLabel)

In [ ]:
useNirmalFunctionEfficiently("Log","none",0,informDFWithLabel)

In [ ]:
useNirmalFunctionEfficiently("Log","none",0,posPatternsDFWithLabel) 

In [ ]:
useNirmalFunctionEfficiently("Log","none",0,contentPostDFWithLabel)

In [ ]:
useNirmalFunctionEfficiently("Log","none",0,uniDFWithLabel)

In [ ]:
#useNirmalFunctionEfficiently("Log","none",0,biDFWithLabel)

# Experiments - find best stemming and whether uni,bi or uni+bi should be used

In [ ]:
#resultsStemming = pd.DataFrame()

In [ ]:
#stemmingChoices = ["porter","snowball","lancaster"]
#maxUni = 542
#maxBig = 152
#classifiers = ["SVM","RandomForest","XgBoost","NaiveBayes"]
#for classifier in classifiers:
#    for pres in [True,False]:
#        for stem in stemmingChoices:
#            for nfeat in [50,100,250,500]:
#                print(stem+"_"+str(nfeat)+"_"+str(pres))
#                if nfeat<maxUni:
#                    uniFeat=getUniAndBigramsFeatures(postTitles,stem,pres,1)
#                    uniFeat.insert(0, "id", TrainingSet["id"].tolist())
#                    uniDFWithLabel = pd.merge(uniFeat,Labels[["id","truthClass"]], on = "id")
#                    [acc1,sd1] =useNirmalFunctionEfficiently(classifier,"f_classif",nfeat,uniDFWithLabel)
 #              
 #               if nfeat<maxBig:
#                    bigFeat=getUniAndBigramsFeatures(postTitles,stem,pres,2)
#                    bigFeat.insert(0, "id", TrainingSet["id"].tolist())
#                    biDFWithLabel = pd.merge(bigFeat,Labels[["id","truthClass"]], on = "id")
#                    [acc2,sd2] =useNirmalFunctionEfficiently(classifier,"f_classif",nfeat,biDFWithLabel)
#
#      
#                allgrams = pd.merge(uniFeat,bigFeat,on="id")
#                allDFWithLabel = pd.merge(allgrams,Labels[["id","truthClass"]], on = "id")#
#
#                [acc3,sd3] =useNirmalFunctionEfficiently(classifier,"f_classif",nfeat,allDFWithLabel)
#                resultsStemming[stem+"_"+str(nfeat)+"_"+str(pres)]=[acc1,acc2,acc3]

#    resultsStemming.to_csv("results"+classifier+"Stemming.csv")

In [ ]:
#Pick classifier for sentiment
classifiers = ["Log","SVM","RandomForest","XgBoost","NaiveBayes"]
for classif in classifiers:
    [acc1,acc2]=useNirmalFunctionEfficiently(classif,"f_classif","all",sentimentWithLabel)
    print(classif+" yields accuracy of:"+str(acc1))

In [ ]:
#Pick classifier for contentpost
classifiers = ["Log","SVM","RandomForest","XgBoost","NaiveBayes"]
for classif in classifiers:
    [acc1,acc2]=useNirmalFunctionEfficiently(classif,"f_classif","all",contentPostDFWithLabel)
    print(classif+" yields accuracy of:"+str(acc1))

In [ ]:
#Pick classifier for informality
classifiers = ["Log","SVM","RandomForest","XgBoost","NaiveBayes"]
for classif in classifiers:
    [acc1,acc2]=useNirmalFunctionEfficiently(classif,"f_classif","all",informDFWithLabel)
    print(classif+" yields accuracy of:"+str(acc1))

In [ ]:
#Try this set all together - since results were so bad
mixtureDFWithLabel = pd.merge(sentiment, contentPostDF, on = "id")
mixtureDFWithLabel = pd.merge(mixtureDFWithLabel,informDF,on="id")
mixtureDFWithLabel = pd.merge(mixtureDFWithLabel, Labels[["id","truthClass"]], on = "id")
mixtureDFWithLabel.to_csv("mixtureDF_AllFeatures_SVM.csv")

In [ ]:
mixtureDFWithLabel

In [ ]:
#Pick classifier for mixture of features
classifiers = ["Log","SVM","RandomForest","XgBoost","NaiveBayes"]
for classif in classifiers:
    [acc1,acc2]=useNirmalFunctionEfficiently(classif,"f_classif","all",mixtureDFWithLabel)
    print(classif+" yields accuracy of:"+str(acc1))

In [ ]:
#This was run with all combiantions of bigram features
#posPatternsDFWithLabel =posPatternsDFWithLabel.loc[:, (posPatternsDFWithLabel != 0).any(axis=0)]
#classifiers = ["Log","SVM","RandomForest","XgBoost","NaiveBayes"]
#resultsposPatterns=pd.DataFrame()
#for classif in classifiers:
#    accuracies=[]
#    for nfeat in [50,100,300,500,700,800]:
#        [acc1,acc2]=useNirmalFunctionEfficiently(classif,"f_classif",nfeat,posPatternsDFWithLabel)
#        accuracies.append(acc1)
#    resultsposPatterns[classif]=accuracies

In [ ]:
#This code was to retrieve the features used in the best setting k= 400
#clf = SVC()
#x = posPatternsDFWithLabel.copy()
#featNorm = x.copy()
#for k in range(len(featNorm.columns)):
#    m = np.mean(x.iloc[:,k])
#    s = np.std(x.iloc[:,k])
#    featNorm.iloc[:,k] = (x.iloc[:,k]-m)/s
    
#featNorm = featNorm.iloc[:,1:k]
#y = x.iloc[:,-1]

#fSel =SelectKBest(f_classif, k=400)
#X_new = fSel.fit_transform(featNorm, y)
# Get idxs of columns to keep
#idxs_selected = fSel.get_support(indices=True)
# Create new dataframe with only desired columns, or overwrite existing
#features= featNorm.columns[idxs_selected]

    #Classifier resuts
#scores = cross_val_score(clf, X_new, y, cv=10)
#accuracy = [scores.mean(), scores.std()]

In [98]:
TrainingSet['postText'].tolist()

[['Should I Get Bings'],
 ['Which TV Female Friend Group Do You Belong In'],
 ['The New "Star Wars: The Force Awakens" Trailer Is Here To Give You Chills'],
 ['This Vine Of New York On "Celebrity Big Brother" Is Fucking Perfect'],
 ['A Couple Did A Stunning Photo Shoot With Their Baby After Learning She Had An Inoperable Brain Tumor'],
 ['How To Flirt With Queer Girls Without Making A Total Fool Of Yourself'],
 ['32 Cute Things To Distract From Your Awkward Thanksgiving'],
 ['If Disney Princesses Were From Florida'],
 ["What's A Quote Or Lyric That Best Describes Your Depression"],
 ['Natalie Dormer And Sam Claflin Play A Game To See How They\'d Actually Last In "The Hunger Games"'],
 ['16 Perfect Responses To The Indian Patriarchy'],
 ['21 Times I Died During The "Captain America: Civil War" Teaser'],
 ['17 Times Kourtney Kardashian Shut Down Her Own Family'],
 ['Does Coffee Make You Poop'],
 ['Who Is Your Celebrity Ex Based On Your Zodiac'],
 ['17 Hairdresser Struggles Every Black Gi